# Qubitization Select

the Select interface and associated bloqs.

Note: we cannot name this module `select` or it breaks the builtin `selectors` module.

In [ ]:
import cirq
import numpy as np
import qualtran
import cirq_ft
import cirq_ft.infra.testing as cq_testing
from qualtran.jupyter_tools import display_gate_and_compilation, show_bloq
from typing import *

## `MySelect`
The SELECT Oracle interface.

The action of a SELECT oracle on a selection register $|l\rangle$ and target register
$|\Psi\rangle$ can be defined as:

$$
    \mathrm{SELECT} = \sum_{l}|l \rangle \langle l| \otimes U_l
$$

In other words, the `SELECT` oracle applies $l$'th unitary $U_{l}$ on the target register
$|\Psi\rangle$ when the selection register stores integer $l$.

$$
    \mathrm{SELECT}|l\rangle |\Psi\rangle = |l\rangle U_{l}|\Psi\rangle
$$

Implementations of `Select` must return `control_registers`, `selection_registers`,
and `system_registers` so that registers can be routed by `BlackBoxSelect`.

In [ ]:
from attrs import frozen

from qualtran import Register, Signature
from qualtran.bloqs.qubitization.select_bloq import Select

@frozen
class MySelect(Select):
    __doc__ = Select.__doc__

    @property
    def control_registers(self) -> List[Register]:
        return []

    @property
    def selection_registers(self) -> List[Register]:
        return list(Signature.build(p=32, q=32, spin=1))

    @property
    def system_register(self) -> Register:
        return Register(name='psi', bitsize=128)

bloq = MySelect()
show_bloq(bloq)

## `BlackBoxSelect`
A 'black box' Select bloq.

The `SELECT` operation applies the $l$'th unitary $U_{l}$ on the system register
when the selection register stores integer $l$.

When implementing specific `Select` bloqs, it is helpful to have multiple selection
registers each with semantic meaning. For example: you could have spatial or spin coordinates
on different, named registers. The `Select` interface encourages this. `BlackBoxSelect`
uses the properties on the `Select` interface to provide a "black box" view of a select
operation that just has a selection and system register.

During decomposition, this bloq will use the `Partition` utility bloq to partition
and route the parts of the unified selection register to the `Select` bloq.

#### Parameters
 - `select`: The bloq implementing the `Select` interface. 

Registers:
 - selection: The combined selection register
 - system: The combined system register

In [ ]:
from qualtran.bloqs.qubitization.select_bloq import BlackBoxSelect, DummySelect

bloq = BlackBoxSelect(DummySelect())
show_bloq(bloq)

In [ ]:
show_bloq(bloq.decompose_bloq())

In [ ]:
c_bbs = bloq.controlled()
show_bloq(c_bbs)

In [ ]:
show_bloq(c_bbs.decompose_bloq())